# CPN generator w/Stopwords

In [1]:
import pandas as pd
import numpy as np
import spacy
import gensim
from collections import Counter 
import sklearn.cluster
import random

random.seed(0)

In [2]:
# data import
data = pd.read_excel(r'../data/Datos Codificados Estudio v2a.xlsx')
model = gensim.models.KeyedVectors.load_word2vec_format( 
    r'../Data/SBW-vectors-300-min5.bin.gz', # using the spanish billion words embeddings
    binary=True # the model is in binary format
)

In [3]:
# Configuracion de spacy
nlp = spacy.load('es_core_news_sm')
# nlp.Defaults.stop_words.add("my_new_stopword")

## Funciones Importantes

In [4]:

def clean_text(text):
    """
    Recive a string and return the string in tokens without punctuations
    and in lowercase
    """
    # for each token in the sentence add to the list if is not a punctuation
    return [t for t in nlp(text.lower()) if not t.is_punct]


def unify_text(tokens):
    """
    Recive al list of string and return in one string without stop words
    """
    # for each token if is not a stop word add the word to the list
    words = [t.orth_ for t in tokens]

    # return the tokens in one string
    return(" ".join(words))


def normalize(tokens):
    """
    Recive al list of string and return in one string without stop words
    """
    # for each token if is not a stop word add the word to the list
    words = [t.orth_ for t in tokens if not t.is_stop]

    # return the tokens in one string
    return(" ".join(words))


def lematize(tokens):
    """
    Recive al list of string and return in one string without stop words 
    and Lematized
    """
    # for each token if is not a stop word add the lemma of the word in the list
    lemmas = [t.lemma_ for t in tokens if not t.is_stop]

    # return the tokens in one string
    return(" ".join(lemmas))

def to_vector(texto,model):
    """ 
    Receives a sentence string along with a word embedding model and 
    returns the vector representation of the sentence
    """
    tokens = texto.split() # splits the text by space and returns a list of words
    vec = np.zeros(300) # creates an empty vector of 300 dimensions
    for word in tokens: # iterates over the sentence
        if word in model: # checks if the word is both in the word embedding
            vec += model[word] # adds every word embedding to the vector
    return vec / np.linalg.norm(vec) # divides the vector by their normal


# Preprocesamiento de los datos

In [5]:
pd.options.mode.chained_assignment = None # Para evitar que se muestre warnings

# Normalizacion de los conceptos, descripciones y codificaciones
for i in range(len(data)):
    data['Descripción'][i] = unify_text(clean_text(data['Descripción'][i]))

## Generación de Matriz de los datos

In [6]:
# Creation of the matrix to make the clustering process
descriptions_matrix = np.zeros( # creatign an empty matrix
    (
        len(data), # the number of data points
        len(model['hola']) # the number of components of the word embedding
    )
)

# matrix filling 
for i,description in enumerate(data['Descripción']):
    vector = to_vector(description,model)
    descriptions_matrix[i,] = vector

# Concatenate the matrix with the data of each observation
data_matrix = np.concatenate([descriptions_matrix,data], axis=1)

#--------------------------------------------------------------------------------------
# This is the important matrix
# Remove of the 'Nan' data
data_matrix_without_nan = data_matrix[~pd.isnull(data_matrix[:,:300]).any(axis=1)]
#--------------------------------------------------------------------------------------

# reduce the matrix to the importan
data_matrix_without_nan = np.concatenate(
    (data_matrix_without_nan[:,:300],                       # Vector of the description
    np.reshape(data_matrix_without_nan[:,302], (-1, 1)),    # Cue/Concept
    np.reshape(data_matrix_without_nan[:,306], (-1, 1)),    # Codification
    np.reshape(data_matrix_without_nan[:,303], (-1, 1))     # Description of the cue
    ),
    axis = 1
)

C:\Users\dra98\AppData\Local\Temp\ipykernel_35564\1988974185.py:53: RuntimeWarning: invalid value encountered in true_divide
  return vec / np.linalg.norm(vec) # divides the vector by their normal


## Model Creation

In [7]:
# Class creation
class PLT_processor:

    def __init__(self, method = 0, n_clusters = 500):
        """
        Recives the type of method to use, 
        - 0: Clasify using K-Means (Simple)
        - 1: Clasify to the codification with the same concept 
        """
        self.n_clusters = n_clusters
        self.method = method # Method of Processing the data
        self.KMeans_dict = {}
        self.KMeans_categories = {}

        if method == 1:
            self.inverse_concept_cluster = {}
            self.concept_cluster = {}

        self.topk = np.zeros((test.shape[0],500))

    def most_frequent(self, List): 
        """
        Recives a list of words, and return the word most frequente of
        the list
        """
        # ? Agregar la frase 'PENDIENTE' en caso de que haya multiples codificaciones con la misma frecuencia ?
        occurence_count = Counter(List) 
        return occurence_count.most_common(1)[0][0] 


    def model_train(self, train):
        """
        Recives the train dataset and the number of clusters to train 
        the k-means model
        """
        
        # If the number of clusters is less than the data points aply K Means
        if(len(train)>self.n_clusters):
            # creates de k-means object
            self.kemean = sklearn.cluster.KMeans(n_clusters=self.n_clusters, random_state=0)
            # Train the k-means algorithm
            self.kemean.fit(train[:,:300])

            # print(self.kemean.n_clusters)
            # Dataframe of train dataset
            df = pd.DataFrame(
                np.concatenate([
                    np.reshape(train[:,301], (-1, 1)),          # Human codification
                    np.reshape(self.kemean.labels_, (-1, 1)),   # Number of the KMean centroid
                    np.reshape(train[:,300], (-1, 1))           # Concept of the codification
                    ], axis=1), 
                columns=['Human', 'KMeans', 'Concept'])
        # Else if the number of clusters is equal than the data points set a number to each point
        elif(len(train)==self.n_clusters):
             # Dataframe of train dataset
            df = pd.DataFrame(
                np.concatenate([
                    np.reshape(train[:,301], (-1, 1)),          # Human codification
                    np.reshape(range(self.n_clusters), (-1, 1)),   # Number of the KMean centroid
                    np.reshape(train[:,300], (-1, 1))           # Concept of the codification
                    ], axis=1), 
                columns=['Human', 'KMeans', 'Concept'])

            self.cluster_centers_2 = train[:,:300].astype(float)

        # create a dictionary of all the humans codifications for each Cluster
        self.KMeans_dict = df.groupby(by='KMeans')['Human'].apply(list).to_dict()

        # Fill a dictionary with the most frecuent value in the centroid
        for key, val in self.KMeans_dict.items():
            self.KMeans_categories[key] = self.most_frequent(val)
        
        # Generates the prediction for the train dataset
        df['KM_Prediction'] = df['KMeans'].map(self.KMeans_categories)

        if self.method == 1:
            # dictionary of the codifications that doesn't show in a concept
            self.not_concept_cluster = {
                i:[False if j in np.unique(df['KMeans'][df['Concept']==i]) else True for j in range(self.n_clusters)] 
                for i in np.unique(df['Concept'])
                }
            # dictionary of clusters for each concept
            self.concept_cluster = {
                i:np.unique(df['KMeans'][df['Concept']==i]) for i in np.unique(df['Concept'])
                }
        # return self.kemean.inertia_

    
    def get_distances(self, test):
        """
        recives the test data to calculate the distances of each frase, return 
        a matrix with the distances sorted
        """

        # Distance matrix of each test point to each cluster center
        distance_matrix = np.zeros((test.shape[0],self.n_clusters))

        if(len(train)>self.n_clusters):
            # for each cluster center
            for i in range(distance_matrix.shape[1]):    
                # Euclidian distance of each point to the i-est cluster center
                distance_matrix[:,i]=np.sqrt(np.sum((test[:,:300].astype(float)-self.kemean.cluster_centers_[i,:])**2,axis=1))
        elif(len(train)==self.n_clusters):
            for i in range(distance_matrix.shape[1]):    
                # Euclidian distance of each point to the i-est cluster center
                distance_matrix[:,i]=np.sqrt(np.sum((test[:,:300].astype(float)-self.cluster_centers_2[i,:])**2,axis=1))
        

        if self.method == 1:
            # for each datapoint
            for i in range(distance_matrix.shape[0]):
                # if the centroid not share the same concept asing NaN value
                distance_matrix[i, self.not_concept_cluster[test[i,300]]] = np.nan
    

        # Sorting distances
        self.topk=np.argsort(distance_matrix,axis=1)

    
    def set_labels(self, test):
        """
        Create a new matrix from the clusters sorted and change the value
        from numeric to the string according the codification
        """

        # Change of the numeric value to the codification 
        self.topKS=pd.DataFrame(self.topk)

        # create a temporal array of the kmeans categories
        tempData = np.array([value for (_, value) in sorted(self.KMeans_categories.items())])
        
        # for each cluster center
        # print("Setting labels")
        # print('len temp',len(self.KMeans_categories))
        # print('dim topKS',self.topKS.shape[1])
        for j in range(self.topKS.shape[1]):
            # set the codification of the numeric value in the topk list
            self.topKS.iloc[:,j]=tempData[self.topk[:,j]]
        
        if self.method == 1:
            # for each datapoint
            for i in range(self.topKS.shape[0]):
                # Remove of the clusters that not share the same concept
                self.topKS.iloc[i, np.isin(self.topk[i,:], self.concept_cluster[test[i,300]] ,invert=True)]= np.nan


    def get_accuracies(self, test):
        """
        Recives the test matrix and return the accuracies of the 
        diferents predictions
        """
        #Creating the accuracy table to check each data point
        testLabel=np.zeros(self.topKS.shape)
        indexes_method0=pd.DataFrame(np.zeros((self.topKS.shape[0],2)), columns=['index', 'value']) 

        #For each data point
        for i in range(testLabel.shape[0]):
            #Checking if some of the cluster is able to classify it right
            boolClass=self.topKS.iloc[i,:]==test[i,301]
            if sum(boolClass)>0:
                getIndex=boolClass.idxmax()
                indexes_method0.iloc[i,0] = getIndex
                indexes_method0.iloc[i,1] = self.topKS.iloc[i,getIndex]
                #Setting the rest of the data point as 1
                testLabel[i,getIndex:]=1
            else:
                indexes_method0.iloc[i,0] = np.nan
                indexes_method0.iloc[i,1] = np.nan
        accuracies=testLabel.sum(axis=0)/testLabel.shape[0]

        return accuracies

    def process(self, train, test):
        self.model_train(train)
        self.get_distances(test)
        self.set_labels(test)
        return self.get_accuracies(test)

    def train_process(self, train):
        self.model_train(train)
        self.get_distances(train)
        self.set_labels(train)
        return self.get_accuracies(train)

# Evaluation

In [8]:
# K = 500
# Top 1, 3, 5, 10, 50
from sklearn.model_selection import KFold

kfold=KFold(n_splits=5)
k500_acc_top = np.zeros((5, 5))

tops=(0,2,4,9,49)

i=0

for train_index, test_index in kfold.split(data_matrix_without_nan):
    train = data_matrix_without_nan[train_index, :]
    test = data_matrix_without_nan[test_index, :]

    proc = PLT_processor(0)
    acc = proc.process(train, test)

    for j in range(len(tops)):
        k500_acc_top[i,j] = acc[tops[j]]

    i+=1

In [9]:
print(k500_acc_top.mean(axis=0))
print(k500_acc_top.std(axis=0))

[0.2539256  0.30059127 0.3195428  0.34970135 0.43019385]
[0.01009957 0.01030224 0.01439319 0.01661072 0.02851408]


In [10]:
# K = 500
# Top 1, 3, 5, 10, 50
from sklearn.model_selection import KFold

kfold=KFold(n_splits=5)
k500_acc_top = np.zeros((5, 5))

tops=(0,2,4,9,49)

i=0

for train_index, test_index in kfold.split(data_matrix_without_nan):
    train = data_matrix_without_nan[train_index, :]
    test = data_matrix_without_nan[test_index, :]

    proc = PLT_processor(0)
    acc = proc.process(train, test)

    for j in range(len(tops)):
        k500_acc_top[i,j] = acc[tops[j]]

    i+=1

In [11]:
print(k500_acc_top.mean(axis=0))
print(k500_acc_top.std(axis=0))

[0.2539256  0.30059127 0.3195428  0.34970135 0.43019385]
[0.01009957 0.01030224 0.01439319 0.01661072 0.02851408]


In [ ]:
# K = 1000
# Top 1, 3, 5, 10, 50
from sklearn.model_selection import KFold

kfold=KFold(n_splits=5)
k1000_acc_top = np.zeros((5, 5))

tops=(0,2,4,9,49)

i=0

for train_index, test_index in kfold.split(data_matrix_without_nan):
    train = data_matrix_without_nan[train_index, :]
    test = data_matrix_without_nan[test_index, :]

    proc = PLT_processor(0, n_clusters=1000)
    acc = proc.train_process(train)

    for j in range(len(tops)):
        k1000_acc_top[i,j] = acc[tops[j]]

    i+=1

In [ ]:
print(k1000_acc_top.mean(axis=0))
print(k1000_acc_top.std(axis=0))

[0.61480993 0.66220763 0.67647398 0.69732477 0.74696931]
[0.00526059 0.00491224 0.00495398 0.00464618 0.00537755]


In [12]:
# K = 1000
# Top 1, 3, 5, 10, 50
from sklearn.model_selection import KFold

kfold=KFold(n_splits=5)
k1000_acc_top = np.zeros((5, 5))

tops=(0,2,4,9,49)

i=0

for train_index, test_index in kfold.split(data_matrix_without_nan):
    train = data_matrix_without_nan[train_index, :]
    test = data_matrix_without_nan[test_index, :]

    proc = PLT_processor(0, n_clusters=1000)
    acc = proc.process(train, test)

    for j in range(len(tops)):
        k1000_acc_top[i,j] = acc[tops[j]]

    i+=1

In [13]:
print(k1000_acc_top.mean(axis=0))
print(k1000_acc_top.std(axis=0))

[0.29672038 0.35459266 0.37884407 0.40859674 0.49989142]
[0.01402193 0.01169362 0.01522491 0.01870023 0.02340218]
